In [1]:
import pandas as pd
import psycopg2

In [2]:
conn = psycopg2.connect("dbname='cap' user='postgres' host='ec2-52-27-114-159.us-west-2.compute.amazonaws.com' port=9000 password ='secret'")
data = pd.read_sql_query("SELECT * FROM articles LIMIT 5", conn)

In [3]:
data.head()

,site,title,author,published_on,accessed_on,url,body,newspaper_keywords,newspaper_summary,id
0,Breitbart,"Trump: ‘We’re Doing Very Well in Iraq,’ U.S. T...",John Hayward,2017-03-29,2017-03-31 08:20:09.478760,http://www.breitbart.com/national-security/201...,"SIGN UP FOR OUR NEWSLETTER On Tuesday, Presid...","{war,told,secretary,soldiers,taking,iraq,presi...","SIGN UP FOR OUR NEWSLETTEROn Tuesday, Presiden...",981
1,Breitbart,Top U.S. General: ‘We Have Not Relaxed the Rul...,Edwin Mora,2017-03-29,2017-03-31 08:20:14.741313,http://www.breitbart.com/national-security/201...,"SIGN UP FOR OUR NEWSLETTER WASHINGTON, D.C. —...","{gen,rules,responsibility,engagement,military,...",We have not relaxed the rules of engagement.\n...,982
2,Breitbart,Protests in Paris Continue for Third Night Aft...,Oliver Jj Lane,2017-03-30,2017-03-31 08:20:18.922523,http://www.breitbart.com/london/2017/03/30/pro...,SIGN UP FOR OUR NEWSLETTER Hundreds of “Asian...,"{night,protest,paris,killed,france,chinese,sub...",SIGN UP FOR OUR NEWSLETTERHundreds of “Asians”...,983
3,Breitbart,Rep. Jim Jordan: Working with Dems on Health C...,Dan Riehl,2017-03-30,2017-03-31 08:20:25.141342,http://www.breitbart.com/radio/2017/03/30/rep-...,SIGN UP FOR OUR NEWSLETTER Rep. Jim Jordan (R...,"{come,jim,healthcare,reform,werent,working,mis...",SIGN UP FOR OUR NEWSLETTERRep. Jim Jordan (R-O...,984
4,Breitbart,John McCain in Last-Minute Attempt to Avert Go...,Ian Mason,2017-03-30,2017-03-31 08:20:29.304708,http://www.breitbart.com/big-government/2017/0...,SIGN UP FOR OUR NEWSLETTER Sen. John McCain (...,"{constitutional,john,supreme,court,attempt,mcc...",SIGN UP FOR OUR NEWSLETTERSen. John McCain (R-...,985


## TF IDF outline 

- list of articles bodies 
- feed list into function to make a sparse matrix for training
- use that with tf-idf stuff above to create a tf-idf transformer 
- use that to label every individual article with tf-idf vectors in a database table


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 10 columns):
site                  5 non-null object
title                 5 non-null object
author                5 non-null object
published_on          5 non-null object
accessed_on           5 non-null datetime64[ns]
url                   5 non-null object
body                  5 non-null object
newspaper_keywords    5 non-null object
newspaper_summary     5 non-null object
id                    5 non-null int64
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 480.0+ bytes


In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# transforms data into tfidf matrix representation
vectorizer = TfidfVectorizer(max_df=0.5, max_features=100,
                                 min_df=2, use_idf=True)


In [6]:
# fit our data (list of article bodies) to a tfidf representation
X = vectorizer.fit_transform(data.body)

# verify we have a sparse matrix of 100 tfidf features for each article 
# should be 5*100 sparse matrix
X

<5x100 sparse matrix of type '<class 'numpy.float64'>'
	with 200 stored elements in Compressed Sparse Row format>

In [7]:
from sklearn.cluster import KMeans

# How many clusters we want
true_k = 3

# create the KMeans object with initial settings
km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1,
                verbose=True)

In [8]:
# fit our tfidf data to the kmeans model
km.fit(X)

Initialization complete
Iteration  0, inertia 2.741
Iteration  1, inertia 1.371
Converged at iteration 1: center shift 0.000000e+00 within tolerance 6.759821e-07


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=3, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=True)

In [9]:
terms = vectorizer.get_feature_names()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

for i in range(3):
    for ind in order_centroids[i, :3]:
            print(' %s' % terms[ind], end='')

 iraq mosul civilian them when its democrats thursday republicans